In [1]:
import pymongo
from pymongo import MongoClient
client = MongoClient()
client = MongoClient('localhost', 27017)
# client = MongoClient('mongodb://localhost:27017/')
db = client.db_cdiscount

In [2]:
import io
import bson                       # this is installed with the pymongo package
import matplotlib.pyplot as plt
from skimage.data import imread   # or, whatever image library you prefer
import multiprocessing as mp      # will come in handy due to the size of the data

In [3]:
import numpy as np

In [6]:
first = 0
last = 169896
batch = 4000
num_batch = int(last/batch)
# last = 82

In [7]:
import time

In [ ]:
start_time = time.time()

counter = 0
row_count = 0
for j in range(0,num_batch-1):
# for j in range(0,1):
    batch_time = time.time()
    a1 = int(list(np.linspace(first,last,num_batch))[j])
    an = int(list(np.linspace(first,last,num_batch))[j+1])
    
    lst_batch = []
    
#     print(a1,an)
    
    cur = db.train.find({})[a1:an]
    print("--- %s read time seconds ---" % (time.time() - batch_time)) 
    m= an-a1
        
    i = 0
#     pic_array
    while (cur.alive):
        idx = cur.next()
        dic = {}
        
        category_id =idx['category_id'] 
#     #     print(category_id)

#         cat = db.cat_encode.find_one({ "cat" : (category_id)}, {"cat" : 1.0, "_id" : 0})['cat']
#     #     print(cat)
        
        picture = np.float32(imread(io.BytesIO(idx['imgs'][0]['picture']))/255.0)
        picture = picture.reshape(180,180,3)
        
        dic['picture'] = picture
        
#         encode = db.cat_encode.find_one({ "cat" : (category_id)}, {"encode" : 1.0, "_id" : 0})['encode']
    #     print(encode)
#         dic['encode'] = encode
        dic['encode'] =idx['category_id'] 
    
        lst_batch.append(dic)
        
        i+=1
    X_batch = np.array([lst['picture'] for lst in lst_batch])
    Y_batch = np.array([lst['encode'] for lst in lst_batch])
#     print('X batch size: ', X_batch.shape)
    print('Y batch size: ', Y_batch.shape)
    
    del(lst_batch)
        
#     model.fit(X_batch, Y_batch, batch_size=batch_size, epochs=epochs, validation_split=0.2)
    
#     print(X_batch.shape[0])
    row_count = row_count + Y_batch.shape[0]
    counter = counter + Y_batch.shape[0]
    
    if(row_count >= 100000):
        print('*************batch count Model save: ',row_count,'*************')
#         row_count = counter + row_count
        model_name = 'model_' + str(counter) +'.h5'
        print(model_name)
#         model.model.save('E://kaggle//Cdiscount//model//'+model_name)
        print('Model saved')
        row_count = 0
    
    print("--- %s seconds ---" % (time.time() - batch_time))
    print('+++++++++++ rows completed: ', counter, '+++++++++++')
    
print("--- %s seconds ---" % (time.time() - start_time))
# model.model.save('E://kaggle//Cdiscount//model//final_model.h5')

--- 0.0 read time seconds ---
Y batch size:  (4143,)
--- 6.457245349884033 seconds ---
+++++++++++ rows completed:  4143 +++++++++++
--- 0.0010001659393310547 read time seconds ---
Y batch size:  (4144,)
--- 52.07289218902588 seconds ---
+++++++++++ rows completed:  8287 +++++++++++
--- 4.636698007583618 read time seconds ---
Y batch size:  (4144,)
--- 75.03351283073425 seconds ---
+++++++++++ rows completed:  12431 +++++++++++
--- 2.1658778190612793 read time seconds ---


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

